In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
  options.add_argument('E:/Programs/geckodriver.exe')
  # options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
i = 0
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [4]:
url = 'https://www.glassdoor.co.in/Job/energy-jobs-SRCH_KO0,6.htm?industryNId=10006'
driver.get(url)

In [5]:
save = 1
job_title = None
company = None
salary = None
location = None 
description = None
posted_on = None
current_date = None 
avg_sal_estimate = None
minrange = None 
maxrange = None 
i = 0

In [12]:
li_elements = driver.find_elements(By.CSS_SELECTOR, '.JobsList_jobsList__Ey2Vo > li')
itirator = len(li_elements)

while i < itirator:
    time.sleep(2)
    data = {}
    try:
        li_elements[i].find_elements(By.CSS_SELECTOR, 'a')[-1].click()
    except:
        i+=1
        continue
    try:
        salary = li_elements[i].find_element(By.CLASS_NAME, 'JobCard_salaryEstimate___m9kY').text.split('\n')[0]
    except:
        i+=1
        continue
    try:
        job_title = ' '.join(driver.find_element(By.CLASS_NAME, 'JobDetails_jobTitle__Rw_gn').text.split('\n'))
    except:
        None 
    try:
        company = li_elements[i].find_element(By.CLASS_NAME, 'EmployerProfile_profileContainer__d5rMb').text.split('\n')[0]
    except:
        None
    try:
        location = li_elements[i].find_element(By.CLASS_NAME, 'JobCard_location__N_iYE').text.split('\n')[0]
    except:
        None 
    try:
        posted_on = li_elements[i].find_element(By.CSS_SELECTOR, ".JobCard_jobCardContainer__l0svv .JobCard_listingAge__KuaxZ").text
    except:
        None
    try:
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".JobDetails_showMoreWrapper__I6uBt .JobDetails_showMore__j5Z_h"))
        )
        show_more_button.click()
    except:
        None
    try:
        description = driver.find_element(By.CSS_SELECTOR, ".JobDetails_jobDescription__6VeBn").text
    except:
        None 
    try:
        avg_sal_estimate = driver.find_element(By.CSS_SELECTOR, ".SalaryEstimate_averageEstimate__xF_7h").text
    except:
        None 
    try:
        sal_range = driver.find_element(By.CSS_SELECTOR, ".SalaryEstimate_salaryRange__TSUO0").text.split('\n')
        minrange = sal_range[1]
        maxrange = sal_range[3]
    except:
        None 
    try:
        temp = {}
        compoverview = driver.find_elements(By.CSS_SELECTOR, ".JobDetails_companyOverviewGrid__CV62w > div")
        for overv in compoverview:
            overv = overv.text.split('\n')
            temp.update({overv[0]:overv[1]})
    except:
        None
    
    current_date = datetime.datetime.now().date()
    data = {
        'job_title':job_title,
        'company_name':company,
        'salary':salary,
        'location':location,
        'description':description,
        'posted_on':posted_on,
        'current_date':current_date,
        'avg_sal_estimate':avg_sal_estimate,
        'minrange':minrange,
        'maxrange':maxrange
    }
    data.update(temp)
    
    if data != {}:
        print(f'\rProgress: {save}', end='')
        save+=1
        tempdf = pd.DataFrame(data, index=[0])
        tempfilename = 'glassdoor_energy_roles.csv'
        save_data_to_disk(tempdf, tempfilename)
        # make all nans.
        tempdf = pd.DataFrame()
    
    job_title = None
    company = None
    salary = None
    location = None 
    description = None
    posted_on = None
    current_date = None 
    avg_sal_estimate = None
    minrange = None 
    maxrange = None   
    
    i += 1
    if i in range(itirator-2, itirator):
        try:
            driver.find_element(By.CSS_SELECTOR, '.button_Button__meEg5').click()
            time.sleep(4)
            itirator = len(li_elements)
        except:
            None 
        try:
            popup = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Modal")))
            if popup.is_displayed():
                driver.find_element(By.CSS_SELECTOR, '.CloseButton').click()
                try:
                    driver.find_element(By.CSS_SELECTOR, '.button_Button__meEg5').click()
                    time.sleep(4)
                    itirator = len(li_elements)
                except:
                    None
        except:
            None         
        time.sleep(5)

Progress: 110

In [13]:
# i = 60
print(i)

181


In [14]:
df = pd.read_csv(r'glassdoor_energy_roles.csv')
subsetcols = df.columns.to_list()
subsetcols.remove('current_date')
subsetcols.remove('posted_on')
df = df[df['salary'] != 'Unpaid']

df.drop_duplicates(subset=subsetcols, inplace=True)
df.reset_index(drop=True,inplace=True)
# df.to_csv('Naukri_raw.csv',index=False)

df.shape

(110, 16)

In [15]:
current_time = datetime.datetime.now()
filename = 'Glassdoor_Energy_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)